# 协程

## 16.1 生成器如何进化成协程

## 16.2 用作协程的生成器的基本行为

In [1]:
def simple_coroutine():
    print('-> coroutine started')
    x = yield
    print('-> coroutine received:', x)

In [2]:
my_coro = simple_coroutine()
my_coro

<generator object simple_coroutine at 0x0000020F2DDE1FC0>

In [3]:
next(my_coro)

-> coroutine started


In [4]:
my_coro.send(42)

-> coroutine received: 42


StopIteration: 

In [5]:
def simple_coroutine(a):
    print('-> Started: a =', a)
    b = yield a
    print('-> Received: b =', b)
    c = yield a + b
    print('-> Received: c =', c)

In [7]:
my_coro2 = simple_coroutine(14)
from inspect import getgeneratorstate
getgeneratorstate(my_coro2)

'GEN_CREATED'

In [8]:
next(my_coro2)

-> Started: a = 14


14

In [9]:
getgeneratorstate(my_coro2)

'GEN_SUSPENDED'

In [10]:
my_coro2.send(28)

-> Received: b = 28


42

In [11]:
my_coro2.send(99)

-> Received: c = 99


StopIteration: 

In [12]:
getgeneratorstate(my_coro2)

'GEN_CLOSED'

## 16.3 示例：使用协程计算移动平均值

In [13]:
def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield average
        total += term
        count += 1
        average = total/count

In [23]:
coro_avg = averager()
next(coro_avg)
coro_avg.send(10)

10.0

In [24]:
coro_avg.send(30)

20.0

In [25]:
coro_avg.send(5)

15.0

## 16.4 使用装饰器自动预激协程

In [27]:
from functools import wraps

def coroutine(func):
    """Decorator: primes `func` by advancing to first `yield`"""
    @wraps(func)
    def primer(*args,**kwargs):
        gen = func(*args,**kwargs)
        next(gen)
        return gen
    return primer

In [28]:
@coroutine
def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield average
        total += term
        count += 1
        average = total/count

In [30]:
coro_avg = averager()
from inspect import getgeneratorstate
getgeneratorstate(coro_avg)

'GEN_SUSPENDED'

In [32]:
coro_avg.send(10)

10.0

In [33]:
coro_avg.send(30)

20.0

In [34]:
coro_avg.send(5)

15.0

## 16.5 终止协程和异常处理

In [36]:
coro_avg = averager()
coro_avg.send(40)

40.0

In [37]:
coro_avg.send(50)

45.0

In [38]:
coro_avg.send('spam')

TypeError: unsupported operand type(s) for +=: 'float' and 'str'

In [39]:
coro_avg.send(60)

StopIteration: 

In [40]:
class DemoException(Exception):
    """An exception type for the demonstration."""

def demo_exc_handling():
    print('-> coroutine started')
    while True:
        try:
            x = yield
        except DemoException:
            print('*** DemoException handled. Continuing...')
        else:
            print('-> coroutine received: {!r}'.format(x))
    raise RuntimeError('This line should never run.')

In [41]:
exc_coro = demo_exc_handling()
next(exc_coro)

-> coroutine started


In [42]:
exc_coro.send(11)

-> coroutine received: 11


In [43]:
exc_coro.send(22)

-> coroutine received: 22


In [44]:
exc_coro.close()

In [49]:
from inspect import getgeneratorstate
getgeneratorstate(exc_coro)

'GEN_CLOSED'

In [50]:
exc_coro = demo_exc_handling()
next(exc_coro)

-> coroutine started


In [51]:
exc_coro.send(11)

-> coroutine received: 11


In [52]:
exc_coro.throw(DemoException)

*** DemoException handled. Continuing...


In [53]:
getgeneratorstate(exc_coro)

'GEN_SUSPENDED'

In [54]:
exc_coro = demo_exc_handling()
next(exc_coro)

-> coroutine started


In [55]:
exc_coro.send(11)

-> coroutine received: 11


In [56]:
exc_coro.throw(ZeroDivisionError)

ZeroDivisionError: 

In [57]:
getgeneratorstate(exc_coro)

'GEN_CLOSED'

In [59]:
class DemoException(Exception):
    """An exception type for the demonstration."""

def demo_exc_handling():
    print('-> coroutine started')
    try:
        while True:
            try:
                x = yield
            except DemoException:
                print('*** DemoException handled. Continuing...')
            else:
                print('-> coroutine received: {!r}'.format(x))
    finally:
        print('-> coroutine ending')

## 16.6 让协程返回值

In [1]:
from collections import namedtuple

Result = namedtuple('Result', 'count average')


def average():
    total = .0
    count = 0
    average = None
    while True:
        term = yield
        if term is None:
            break
        total += term
        count += 1
        average = total/count
    return Result(count, average)

In [2]:
coro_avg = average()
next(coro_avg)
coro_avg.send(10)
coro_avg.send(30)
coro_avg.send(6.5)
coro_avg.send(None)

StopIteration: Result(count=3, average=15.5)

In [3]:
coro_avg = average()
next(coro_avg)
coro_avg.send(10)
coro_avg.send(30)
coro_avg.send(6.5)
try:
    coro_avg.send(None)
except StopIteration as exc:
    result = exc.value
result

Result(count=3, average=15.5)

## 16.7 使用yield from

In [4]:
def gen():
    for c in 'AB':
        yield c
    for i in range(1,3):
        yield i

list(gen())

['A', 'B', 1, 2]

In [5]:
def gen():
    yield from 'AB'
    yield from range(1,3)

list(gen())

['A', 'B', 1, 2]

In [6]:
def chain(*iterables):
    for it in iterables:
        yield from it

In [7]:
s = 'ABC'
t = tuple(range(3))
list(chain(s,t))

['A', 'B', 'C', 0, 1, 2]

In [10]:
from collections import namedtuple

Result = namedtuple('Result', 'count average')

def average():
    total = .0
    count = 0
    average = None
    while True:
        term = yield
        if term is None:
            break
        total += term
        count += 1
        average = total/count
    return Result(count, average)


def grouper(results, key):
    while True:
        results[key] = yield from average()

    
def main(data):
    results = {}
    for key, values in data.items():
        group = grouper(results, key)
        next(group)
        for value in values:
            group.send(value)
        group.send(None)
    print(results)
    report(results)


def report(results):
    for key, result in sorted(results.items()):
        group, unit = key.split(';')
        print('{:2} {:5} averaging {:.2f}{}'.format(result.count, group, result.average, unit)) 


In [11]:
data = {
    'girls;kg':
        [40.9, 38.5, 44.3, 42.2, 45.2, 41.7, 44.5, 38.0, 40.6, 44.5],
    'girls;m':
        [1.6, 1.51, 1.4, 1.3, 1.41, 1.39, 1.33, 1.45, 1.43, 1.34],
    'boys;kg':
        [39.0, 40.8, 43.2, 40.8, 43.1, 38.6, 41.4, 40.6, 36.3],
    'boys;m':
        [1.38, 1.5, 1.32, 1.5, 1.3, 1.41, 1.39, 1.33, 1.45]
}

main(data)

{'girls;kg': Result(count=10, average=42.040000000000006), 'girls;m': Result(count=10, average=1.416), 'boys;kg': Result(count=9, average=40.422222222222224), 'boys;m': Result(count=9, average=1.3977777777777778)}
 9 boys  averaging 40.42kg
 9 boys  averaging 1.40m
10 girls averaging 42.04kg
10 girls averaging 1.42m
